# MCP问题

M*N问题：没有标准的情况下M个ai应用连接到N个外部工具的挑战

Host（ai应用层），client（Mcp-Client负责连接各种server并执行对应服务得到结果返回llm），server（提供各种服务）

## 通讯

### 通讯协议
MCP的通讯协议：JSON-RPC（基于json的轻量远程调用协议），该协议定义了三种类型的消息

1. 请求，client到server的请求，包括id，调用的方法，参数
2. 响应，server到client的返回，包括请求对应的id，result结果
3. 通知，server到client的，一般是提供有关事件的更新或通知

### 通讯机制
client和server的通讯机制

1. 本地stdio输入输出，host将server作为子进程启动，通过stdin和stdout通信
2. Http+SSE通信，SSE是服务器发送事件

### 交互流程

1. 初始化：client连接到server，交换协议版本和功能 --》 server响应支持的版本 --》Client通知初始化完成
2. 发现：client请求可用功能信息 --》 server响应可用工具的列表
3. 执行：client请求某个功能呢 --》 server可以返回通知，server也可以返回执行结果响应
4. 终止：client发送终止信息 --》 server响应确认 --》 client发送最终退出消息

## MCP组件

### 1. 工具

类似langchain的tool，一段工作函数，执行某个任务，比如查询天气。由llm决定什么时候调用这些工具

### 2. 资源

对数据源的访问，只读的，比如查看文件内容，检索数据库记录，读取配置

### 3. 提示词

server到client，预定义的提示词模板，用于llm和server互动来引导工作流程

### 4. 采样

复杂任务时，是server请求client的llm帮忙做某件事，一般用于多任务步骤

### 以上的组件发现

client连接到server时通过特定list查询提供的工具，资源，提示词等

1. tools/list
2. resources/list
3. prompts/list



## MCP的Server端开发工具SDK



In [ ]:
# 这里我使用了UV管理器来管理python下的MCP项目
# 初始化完成后编写server.py文件
# 最后启动server：uv run mcp dev server.py
from mcp.server.fastmcp import FastMCP

# Create an MCP server
mcp = FastMCP("Weather Service")

# Tool implementation
@mcp.tool()
def get_weather(location: str) -> str:
    """Get the current weather for a specified location."""
    return f"Weather in {location}: Sunny, 72°F"

# Resource implementation
@mcp.resource("weather://{location}")
def weather_resource(location: str) -> str:
    """Provide weather data as a resource."""
    return f"Weather data for {location}: Sunny, 72°F"

# Prompt implementation
@mcp.prompt()
def weather_report(location: str) -> str:
    """Create a weather report prompt."""
    return f"""You are a weather reporter. Weather report for {location}?"""


# Run the server
if __name__ == "__main__":
    mcp.run()

## MCP客户端

客户端负责和mcp服务器进行交互

Anthropic 的 Claude Desktop 是最著名的 MCP 客户端之一，提供与各种 MCP 服务器的集成
